In [1]:
import pandas as pd
import numpy as np
from datetime import datetime         
from sklearn import preprocessing

### Cargo datos desactualizados

In [2]:
data_set_complete = pd.read_csv('data_set_complete.csv', na_values={'sexo':'NO_DECLARA'}, low_memory=False)
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13208286 entries, 0 to 13208285
Data columns (total 16 columns):
sepostulo               int64
idaviso                 int64
idpostulante            object
fechapostulacion        object
ultima_vista            object
veces_visto             int64
estudios                object
estado                  object
sexo                    object
edad                    int64
titulo                  object
nombre_zona             object
tipo_de_trabajo         object
nivel_laboral           object
nombre_area             object
denominacion_empresa    object
dtypes: int64(4), object(12)
memory usage: 1.6+ GB


In [3]:
#FIltro aquellos con campos nulos, total tengo muchas muestras
data_set_complete.dropna(subset=['sexo','edad','nombre_zona','tipo_de_trabajo','nivel_laboral','nombre_area'], inplace=True)

data_set_complete.drop(columns=['estudios','estado'], inplace=True)

data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11870121 entries, 0 to 13208285
Data columns (total 14 columns):
sepostulo               int64
idaviso                 int64
idpostulante            object
fechapostulacion        object
ultima_vista            object
veces_visto             int64
sexo                    object
edad                    int64
titulo                  object
nombre_zona             object
tipo_de_trabajo         object
nivel_laboral           object
nombre_area             object
denominacion_empresa    object
dtypes: int64(4), object(10)
memory usage: 1.3+ GB


### Concateno toda la info de educacion

In [4]:
edu_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')])
edu_totales.drop_duplicates(inplace=True)

edu_totales = edu_totales.rename(columns={'nombre':'estudios'})
edu_totales['estudios'] = edu_totales['estudios'].astype('category')
edu_totales['estado'] = edu_totales['estado'].astype('category')

#Saco abandonados y me separo graduados de en curso

edu_graduados = edu_totales[edu_totales['estado']=='Graduado'].drop(columns='estado')
edu_encurso = edu_totales[edu_totales['estado']=='En Curso'].drop(columns='estado')

del edu_totales

# Creo nuevas columnas indicando el orden del nivel del estudio

edu_estudios_list = ['Ninguno','Secundario', 'Otro', 'Terciario/Técnico', 'Universitario', 'Posgrado', 'Master', 'Doctorado']

edu_graduados['estudio_graduado'] = edu_graduados['estudios'].apply(lambda x: edu_estudios_list.index(x)).astype('int8')
edu_graduados.sort_values('estudio_graduado', ascending=True, inplace=True)
edu_graduados.drop_duplicates(subset=['idpostulante'], keep='last', inplace=True)
edu_graduados.sort_values(by=['idpostulante'], ascending=True, inplace=True)
edu_graduados.rename(columns={'estudios':'graduado'}, inplace=True)
edu_graduados.reset_index(drop=True, inplace=True)

edu_encurso['estudio_encurso'] = edu_encurso['estudios'].apply(lambda x: edu_estudios_list.index(x)).astype('int8')
edu_encurso.sort_values('estudio_encurso', ascending=True, inplace=True)
edu_encurso.drop_duplicates(subset=['idpostulante'], keep='last', inplace=True)
edu_encurso.sort_values(by=['idpostulante'], ascending=True, inplace=True)
edu_encurso.rename(columns={'estudios':'encurso'}, inplace=True)
edu_encurso.reset_index(drop=True, inplace=True)

edu_totales = pd.merge(edu_encurso, edu_graduados, how='outer', on='idpostulante')

del edu_graduados, edu_encurso

edu_totales.head()

,idpostulante,encurso,estudio_encurso,graduado,estudio_graduado
0,0zB028M,Terciario/Técnico,3.0,Otro,2.0
1,0zB03O9,Terciario/Técnico,3.0,Secundario,1.0
2,0zB0PLa,Universitario,4.0,Secundario,1.0
3,0zB0WZE,Universitario,4.0,NaN,NaN
4,0zB0Xwd,Universitario,4.0,Secundario,1.0


### Concateno todos los set de vistas y calculo cantidad de vistas de cada usuario a cada anuncio

In [5]:
vistas_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_3_vistas.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_3_vistas.csv')])

def normalize_str(date_str):
    return (date_str[:10]+' '+date_str[11:19])
    
vistas_totales['timestamp'] = vistas_totales['timestamp'].apply(normalize_str)
vistas_totales['timestamp'] = pd.to_datetime(vistas_totales['timestamp'])

vistas_totales.drop_duplicates(inplace=True)

vistas_totales.rename(columns={'idAviso':'idaviso'}, inplace=True)

#Calculo cantidad de vistas de cada usuario a cada anuncio

vistas_totales_count = vistas_totales.groupby(by=['idaviso', 'idpostulante']).count()
vistas_totales_count.reset_index(drop=False, inplace=True)
vistas_totales_count.rename(columns={'timestamp':'veces_visto'}, inplace=True)

del vistas_totales

vistas_totales_count.head()

,idaviso,idpostulante,veces_visto
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


### Incorporo feature cuantas postulaciones totales tuvo ese anuncio, una suerte de "popularidad del anuncio en terminos de postulaciones"

In [ ]:
post_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')])
post_totales.drop(columns='fechapostulacion', inplace=True)
post_totales.drop_duplicates(inplace=True)
post_totales.reset_index(drop=True, inplace=True)

post_por_avisos = post_totales.groupby(by='idaviso').count()
post_por_avisos.rename(columns={'idpostulante':'post_x_aviso'},inplace=True)
post_por_avisos.sort_values(by=['post_x_aviso'], ascending=False, inplace=True)
post_por_avisos.reset_index(drop=False, inplace=True)

del post_totales

post_por_avisos.head()

### Incorporo feature cuantas vistas totales tuvo ese anuncio, una suerte de "popularidad del anuncio en terminos de vistas"

In [ ]:
vistas_totales = pd.concat([pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv'), pd.read_csv('fiuba_hasta_15_abril/fiuba_3_vistas.csv'), pd.read_csv('Fiuba desde 15 Abril/fiuba_3_vistas.csv')])
vistas_totales.drop(columns='timestamp', inplace=True)
vistas_totales.drop_duplicates(inplace=True)
vistas_totales.rename(columns={'idAviso':'idaviso'}, inplace=True)

vistas_por_avisos = vistas_totales.groupby(by='idaviso').count()
vistas_por_avisos.rename(columns={'idpostulante':'vistas_x_aviso'},inplace=True)
vistas_por_avisos.sort_values(by=['vistas_x_aviso'], ascending=False, inplace=True)
vistas_por_avisos.reset_index(drop=False, inplace=True)

del vistas_totales

vistas_por_avisos.head()

### Trabajo sobre la data

In [6]:
data_set_complete = data_set_complete.merge(edu_totales[['idpostulante','estudio_encurso','estudio_graduado']], on='idpostulante', how='left')

data_set_complete['estudio_encurso'] = data_set_complete['estudio_encurso'].fillna(value=0).astype('int8')
data_set_complete['estudio_graduado'] = data_set_complete['estudio_graduado'].fillna(value=0).astype('int8')

In [7]:
data_set_complete['tipo_de_trabajo'].value_counts()

Full-time          10065447
Part-time           1621257
Pasantia              57830
Por Horas             42182
Temporario            31959
Fines de Semana       17557
Por Contrato          16748
Teletrabajo           12483
Primer empleo          4161
Voluntario              497
Name: tipo_de_trabajo, dtype: int64

In [8]:
data_set_complete['tipo_full-time'] = data_set_complete['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Full-time') else 0).astype('int8')
data_set_complete['tipo_part-time'] = data_set_complete['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Part-time') else 0).astype('int8')
data_set_complete['tipo_otro'] = data_set_complete['tipo_de_trabajo'].apply(lambda x: 1 if ((x != 'Full-time') & (x != 'Part-time')) else 0).astype('int8')

In [9]:
data_set_complete['nombre_zona'].value_counts()

Gran Buenos Aires    11050092
Capital Federal        816960
GBA Oeste                3069
Name: nombre_zona, dtype: int64

In [10]:
data_set_complete['zona_gba'] = data_set_complete['nombre_zona'].apply(lambda x: 1 if ((x=='Gran Buenos Aires') or (x=='GBA Oeste')) else 0).astype('int8')
data_set_complete['zona_caba'] = data_set_complete['nombre_zona'].apply(lambda x: 1 if (x=='Capital Federal') else 0).astype('int8')

data_set_complete.drop(columns='nombre_zona', inplace=True)

In [11]:
data_set_complete['nivel_laboral'].value_counts()

Senior / Semi-Senior                    7746295
Junior                                  2675548
Otro                                     993384
Jefe / Supervisor / Responsable          367077
Gerencia / Alta Gerencia / Dirección      87817
Name: nivel_laboral, dtype: int64

In [12]:
data_set_complete['nivel_senior'] = data_set_complete['nivel_laboral'].apply(lambda x: 1 if (x=='Senior / Semi-Senior') else 0).astype('int8')
data_set_complete['nivel_junior'] = data_set_complete['nivel_laboral'].apply(lambda x: 1 if (x=='Junior') else 0).astype('int8')
data_set_complete['nivel_otro'] = data_set_complete['nivel_laboral'].apply(lambda x: 1 if (x=='Otro') else 0).astype('int8')
data_set_complete['nivel_jefe'] = data_set_complete['nivel_laboral'].apply(lambda x: 1 if (x=='Jefe / Supervisor / Responsable') else 0).astype('int8')
data_set_complete['nivel_gerente'] = data_set_complete['nivel_laboral'].apply(lambda x: 1 if (x=='Gerencia / Alta Gerencia / Dirección') else 0).astype('int8')

In [13]:
data_set_complete['sexo'] = data_set_complete['sexo'].apply(lambda x: 1 if (x=='MASC') else 0).astype('int8')

- Vuelvo a calcular y a mergear las veces_visto de cada user a cada anuncio, ya que en inexistentes no vamos a suponer 1 vista default a los postulantes porque overfitea

In [14]:
data_set_complete['veces_visto'] = (pd.merge(data_set_complete[['idaviso','idpostulante']], vistas_totales_count, on=['idaviso', 'idpostulante'], how='left'))['veces_visto']
data_set_complete['veces_visto'].fillna(value=0, inplace=True)
data_set_complete['veces_visto'] = data_set_complete['veces_visto'].astype('int16')
data_set_complete.reset_index(drop=True, inplace=True)

In [ ]:
data_set_complete = data_set_complete.merge(post_por_avisos, how='left', on='idaviso')

if(data_set_complete['post_x_aviso'].hasnans):
    data_set_complete['post_x_aviso'].fillna(value=0, inplace=True)

data_set_complete['post_x_aviso'] = data_set_complete['post_x_aviso'].astype(int)

In [ ]:
data_set_complete = data_set_complete.merge(vistas_por_avisos, how='left', on='idaviso')

if(data_set_complete['vistas_x_aviso'].hasnans):
    data_set_complete['vistas_x_aviso'].fillna(value=0, inplace=True)

data_set_complete['vistas_x_aviso'] = data_set_complete['vistas_x_aviso'].astype(int)

In [15]:
data_set_complete.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,sexo,edad,titulo,tipo_de_trabajo,...,tipo_full-time,tipo_part-time,tipo_otro,zona_gba,zona_caba,nivel_senior,nivel_junior,nivel_otro,nivel_jefe,nivel_gerente
0,1,1112384041,0z5Dmrd,2018-03-28 16:24:37,2018-03-28 15:58:15,1,1,52,Gerente de RRHH- Para Importante Cadena de Far...,Full-time,...,1,0,0,1,0,1,0,0,0,0
1,1,1112420060,0z5Dmrd,2018-03-28 16:47:11,2018-03-28 16:25:03,1,1,52,Coordinador/a de RRHH,Full-time,...,1,0,0,1,0,1,0,0,0,0
2,1,1112428361,0z5JW1r,2018-04-01 07:26:12,2018-04-01 07:23:35,2,1,47,Gerente Nacional de Operaciones de Tiendas - R...,Full-time,...,1,0,0,1,0,0,0,0,0,1
3,1,1112309728,0z5JW1r,2018-03-02 03:49:53,NaN,0,1,47,Asesor de Franquicias Gastronómicas,Full-time,...,1,0,0,1,0,1,0,0,0,0
4,1,1112292169,0z5JW1r,2018-02-19 05:12:24,NaN,0,1,47,Gerente de Operaciones de Franquicias ( Zona N...,Full-time,...,1,0,0,1,0,0,0,0,0,1


In [16]:
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11870121 entries, 0 to 11870120
Data columns (total 25 columns):
sepostulo               int64
idaviso                 int64
idpostulante            object
fechapostulacion        object
ultima_vista            object
veces_visto             int16
sexo                    int8
edad                    int64
titulo                  object
tipo_de_trabajo         object
nivel_laboral           object
nombre_area             object
denominacion_empresa    object
estudio_encurso         int8
estudio_graduado        int8
tipo_full-time          int8
tipo_part-time          int8
tipo_otro               int8
zona_gba                int8
zona_caba               int8
nivel_senior            int8
nivel_junior            int8
nivel_otro              int8
nivel_jefe              int8
nivel_gerente           int8
dtypes: int16(1), int64(3), int8(13), object(8)
memory usage: 1.1+ GB


In [17]:
#data_set_complete.to_csv('data_set_complete_v2.csv', index=False)

### Actualizo tambien la info del test100k

In [20]:
test100k = pd.read_csv('test_final_100k_complete.csv', low_memory=False)

aux = ['MASC', 'FEM']
test100k['sexo'] = test100k['sexo'].apply(lambda x: aux[np.random.randint(2)] if (x == 'NO_DECLARA') else x)
del aux

test100k.drop(columns=['estudios','estado'], inplace=True)

In [30]:
test100k['sexo'].hasnans, test100k['nombre_zona'].hasnans, test100k['tipo_de_trabajo'].hasnans, test100k['nivel_laboral'].hasnans, test100k['nombre_area'].hasnans

(False, False, True, False)

In [32]:
test100k['nivel_laboral'].fillna(value='Senior / Semi-Senior',inplace=True)

In [34]:
test100k = test100k.merge(edu_totales[['idpostulante','estudio_encurso','estudio_graduado']], on='idpostulante', how='left')

test100k['estudio_encurso'] = test100k['estudio_encurso'].fillna(value=0).astype('int8')
test100k['estudio_graduado'] = test100k['estudio_graduado'].fillna(value=0).astype('int8')

In [37]:
test100k['tipo_full-time'] = test100k['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Full-time') else 0).astype('int8')
test100k['tipo_part-time'] = test100k['tipo_de_trabajo'].apply(lambda x: 1 if (x=='Part-time') else 0).astype('int8')
test100k['tipo_otro'] = test100k['tipo_de_trabajo'].apply(lambda x: 1 if ((x != 'Full-time') & (x != 'Part-time')) else 0).astype('int8')

In [43]:
test100k['nombre_zona'].value_counts()

Gran Buenos Aires              88258
Capital Federal                11065
Buenos Aires (fuera de GBA)      408
Cordoba                          119
GBA Oeste                         79
La Plata                          13
Ciudad de Mendoza                 11
Corrientes                         9
Mendoza                            9
Rosario                            7
San Juan                           4
Santa Cruz                         4
Tucuman                            4
Santa Fe                           4
Neuquen                            3
Catamarca                          3
Name: nombre_zona, dtype: int64

In [45]:
test100k['zona_gba'] = test100k['nombre_zona'].apply(lambda x: 1 if (x != 'Capital Federal') else 0).astype('int8')
test100k['zona_caba'] = test100k['nombre_zona'].apply(lambda x: 1 if (x == 'Capital Federal') else 0).astype('int8')

test100k.drop(columns='nombre_zona', inplace=True)

In [46]:
test100k['nivel_senior'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Senior / Semi-Senior') else 0).astype('int8')
test100k['nivel_junior'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Junior') else 0).astype('int8')
test100k['nivel_otro'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Otro') else 0).astype('int8')
test100k['nivel_jefe'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Jefe / Supervisor / Responsable') else 0).astype('int8')
test100k['nivel_gerente'] = test100k['nivel_laboral'].apply(lambda x: 1 if (x=='Gerencia / Alta Gerencia / Dirección') else 0).astype('int8')

In [47]:
test100k['sexo'] = test100k['sexo'].apply(lambda x: 1 if (x=='MASC') else 0).astype('int8')

In [ ]:
test100k['veces_visto'] = (pd.merge(test100k[['idaviso','idpostulante']], vistas_totales_count, on=['idaviso', 'idpostulante'], how='left'))['veces_visto']
test100k['veces_visto'].fillna(value=0, inplace=True)
test100k['veces_visto'] = test100k['veces_visto'].astype('int16')
test100k.reset_index(drop=True, inplace=True)

In [ ]:
test100k = test100k.merge(post_por_avisos, how='left', on='idaviso')

if(test100k['post_x_aviso'].hasnans):
    test100k['post_x_aviso'].fillna(value=0, inplace=True)

test100k['post_x_aviso'] = test100k['post_x_aviso'].astype(int)

In [ ]:
test100k = test100k.merge(vistas_por_avisos, how='left', on='idaviso')

if(test100k['vistas_x_aviso'].hasnans):
    test100k['vistas_x_aviso'].fillna(value=0, inplace=True)

test100k['vistas_x_aviso'] = test100k['vistas_x_aviso'].astype(int)

In [48]:
test100k.head()

,id,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,sexo,edad,titulo,tipo_de_trabajo,...,tipo_full-time,tipo_part-time,tipo_otro,zona_gba,zona_caba,nivel_senior,nivel_junior,nivel_otro,nivel_jefe,nivel_gerente
0,0,739260,6M9ZQR,NaN,NaN,0,0,42,Asistente Comercial,Full-time,...,1,0,0,0,1,0,0,0,1,0
1,1,739260,6v1xdL,NaN,NaN,0,1,30,Asistente Comercial,Full-time,...,1,0,0,0,1,0,0,0,1,0
2,2,739260,ezRKm9,NaN,NaN,0,0,36,Asistente Comercial,Full-time,...,1,0,0,0,1,0,0,0,1,0
3,3,758580,1Q35ej,NaN,NaN,0,1,68,Oracle Financials Sr. Application Developer,Full-time,...,1,0,0,0,1,0,0,1,0,0
4,4,758580,EAN4J6,NaN,NaN,0,0,32,Oracle Financials Sr. Application Developer,Full-time,...,1,0,0,0,1,0,0,1,0,0


In [49]:
test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
fechapostulacion        0 non-null float64
ultima_vista            34161 non-null object
veces_visto             100000 non-null int16
sexo                    100000 non-null int8
edad                    100000 non-null int64
titulo                  100000 non-null object
tipo_de_trabajo         100000 non-null object
nivel_laboral           100000 non-null object
nombre_area             100000 non-null object
denominacion_empresa    99988 non-null object
estudio_encurso         100000 non-null int8
estudio_graduado        100000 non-null int8
tipo_full-time          100000 non-null int8
tipo_part-time          100000 non-null int8
tipo_otro               100000 non-null int8
zona_gba                100000 non-null int8
zona_caba            

In [51]:
#test100k.to_csv('test100k_complete_v2.csv', index=False)